In [14]:
#!pip install transformers
#! pip install torch
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 40.5 MB/s eta 0:00:00


In [21]:
# necessary imports

import pandas as pd
import faiss
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch

In [42]:
# loading test data

user_data = [
    ["13.12.2023 17:14:56", 4.76, 5.04, 4.48, 3.64, 4.48, 5.32],
    ["13.12.2023 17:21:03", 3.64, 3.92, 3.92, 3.64, 3.64, 3.92],
    ["13.12.2023 17:47:13", 5.88, 6.44, 6.65, 7.00, 5.60, 5.32],
    ["13.12.2023 18:03:34", 2.52, 1.40, 5.88, 2.24, 4.20, 1.96],
    ["13.12.2023 18:05:36", 5.88, 7.00, 5.04, 4.76, 7.00, 4.76],
    ["13.12.2023 18:09:50", 3.36, 4.76, 7.00, 3.64, 3.36, 4.48],
    ["13.12.2023 18:42:14", 6.44, 5.88, 3.64, 3.64, 4.76, 5.32],
    ["13.12.2023 19:18:19", 4.76, 5.04, 2.24, 4.48, 3.36, 4.20],
    ["13.12.2023 21:09:46", 5.04, 6.16, 6.44, 5.88, 5.88, 5.60],
    ["16.12.2023 11:36:37", 4.20, 3.92, 3.92, 4.48, 4.48, 5.04],
    ["16.12.2023 13:02:24", 5.88, 7.00, 7.00, 7.00, 7.00, 7.00],
    ["16.12.2023 13:07:36", 4.20, 4.76, 4.48, 4.20, 4.20, 4.76],
    ["16.12.2023 16:12:31", 5.04, 7.00, 3.92, 5.60, 7.00, 6.72],
    ["16.12.2023 16:16:46", 4.76, 6.44, 4.76, 4.76, 7.00, 5.32],
    ["16.12.2023 16:17:51", 6.44, 6.16, 4.48, 5.04, 5.04, 5.88],
    ["16.12.2023 16:33:55", 4.20, 6.72, 7.00, 3.36, 4.76, 4.76],
    ["16.12.2023 18:38:24", 3.92, 4.20, 4.48, 5.88, 5.88, 4.20],
    ["16.12.2023 18:58:30", 4.20, 5.32, 5.88, 5.32, 4.48, 5.60],
    ["17.12.2023 09:45:31", 5.32, 6.44, 5.32, 3.64, 2.80, 6.72],
    ["17.12.2023 09:48:34", 5.32, 4.20, 1.96, 1.96, 3.08, 3.92],
    ["17.12.2023 22:22:43", 4.20, 5.32, 4.48, 5.04, 5.04, 5.88],
    ["17.12.2023 23:17:10", 5.04, 6.44, 5.32, 6.44, 5.88, 5.60],
    ["18.12.2023 14:25:16", 4.48, 4.76, 3.64, 4.48, 4.76, 5.04],
    ["23.12.2023 07:38:09", 3.64, 5.88, 2.52, 3.08, 4.76, 5.60],
]

columns = ["Timestamp", "Realistic", "Investigative", "Artistic", "Social", "Enterprising", "Conventional"]

user_data = pd.DataFrame(user_data, columns=columns)

#print(user_data)

print(np.array(user_data.iloc[0, 1:]))


[4.76 5.04 4.48 3.64 4.48 5.32]


In [47]:
def preprocess_to_faiss(df):

    # normalize
    df = df / np.linalg.norm(df, axis=1, keepdims=True)

    index = faiss.IndexFlatIP(df.shape[1]) # Inner Product for cosine similarity)
    index.add(df)

    return index


def find_similar_with_faiss(user_riasec, index, top_k=100): # quickly sort and retrieve top 100 results

    # normalize
    user_riasec = user_riasec / np.linalg.norm(user_riasec)

    # top_k most similar professions
    distances, indices = index.search(user_riasec.reshape(1, -1), top_k)

    return distances.flatten(), indices.flatten()



In [50]:
# loading riasec data

jd = pd.read_excel("Processed_Interests.xlsx")

print(jd.head)

riasec_columns = ["R_score", "I_score", "A_score", "S_score", "E_score", "C_score"]
processed_jd = jd[riasec_columns]

# Preprocess tto the faiss
jd_faiss = preprocess_to_faiss(processed_jd)

# get similar professions
distances, indices = find_similar_with_faiss(np.array(user_data.iloc[0, 1:]), jd_faiss, top_k=100)
answer = np.column_stack((jd.iloc[indices, 0], distances))

print(answer)



<bound method NDFrame.head of                                     profession  R_score  I_score  A_score  \
0                             Chief Executives     1.33     2.00     2.67   
1                Chief Sustainability Officers     1.00     4.33     2.67   
2              General and Operations Managers     1.33     1.33     1.00   
3                                  Legislators     1.00     3.67     3.67   
4          Advertising and Promotions Managers     1.67     2.00     5.33   
..                                         ...      ...      ...      ...   
969    Pump Operators, Except Wellhead Pumpers     7.00     4.00     1.00   
970                           Wellhead Pumpers     7.00     3.67     1.00   
971  Refuse and Recyclable Material Collectors     7.00     1.33     1.00   
972                 Mine Shuttle Car Operators     7.00     1.33     1.00   
973          Tank Car, Truck, and Ship Loaders     7.00     3.00     1.00   

     S_score  E_score  C_score  HP_1  HP_2  H